### __CNN Assignment | Mohammed Asif Sahadh - 24MSD7061__

#### Import libraries

In [2]:
import numpy as np
import os
from PIL import Image

#### Training data - Bears

In [3]:
train_bears_dir = r"pandas-bears\1. train\bears"
train_bears_imgs = []
for img in os.listdir(train_bears_dir):
    img_path = os.path.join(train_bears_dir, img)
    train_bears_img = Image.open(img_path, 'r')
    train_bears_imgs.append(train_bears_img)

train_bears_data = np.array(train_bears_imgs)

#### Training data - Pandas

In [4]:
train_pandas_dir = r"pandas-bears\1. train\pandas"
train_pandas_imgs = []
for img in os.listdir(train_pandas_dir):
    img_path = os.path.join(train_pandas_dir, img)
    train_pandas_img = Image.open(img_path, 'r')
    train_pandas_imgs.append(train_pandas_img)

train_pandas_data = np.array(train_pandas_imgs)

#### Testing data - Bears

In [5]:
test_bears_dir = r"pandas-bears\2. test\bears"
test_bears_imgs = []
for img in os.listdir(test_bears_dir):
    img_path = os.path.join(test_bears_dir, img)
    test_bears_img = Image.open(img_path, 'r')
    test_bears_imgs.append(test_bears_img)

test_bears_data = np.array(test_bears_imgs)

#### Testing data - Pandas

In [6]:
test_pandas_dir = r"pandas-bears\2. test\pandas"
test_pandas_imgs = []
for img in os.listdir(test_pandas_dir):
    img_path = os.path.join(test_pandas_dir, img)
    test_pandas_img = Image.open(img_path, 'r')
    test_pandas_imgs.append(test_pandas_img)

test_pandas_data = np.array(test_pandas_imgs)

#### Combine

In [7]:
X_train = np.concatenate((train_pandas_data, train_bears_data), axis = 0)
X_test = np.concatenate((test_pandas_data, test_bears_data), axis = 0)

X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

_Let Pandas be 0 & Bears be 1._

In [8]:
train_lab_p = np.zeros(train_pandas_data.shape[0])
train_lab_b = np.ones(train_bears_data.shape[0])

test_lab_p = np.zeros(test_pandas_data.shape[0])
test_lab_b = np.zeros(test_bears_data.shape[0])

In [9]:
y_train = np.concatenate((train_lab_p, train_lab_b), axis = 0)
y_test = np.concatenate((test_lab_p, test_lab_b), axis = 0)

#### Convert np array to torch tensor

In [10]:
import torch

#### Get GPU

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [12]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()

y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [13]:
y_train.shape

torch.Size([500])

In [14]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [15]:
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")
print()
print(f"Training labels shape: {y_train.shape}")
print(f"Testing labels shape: {y_test.shape}")

Training data shape: torch.Size([500, 3, 256, 256])
Testing data shape: torch.Size([100, 3, 256, 256])

Training labels shape: torch.Size([500])
Testing labels shape: torch.Size([100])


#### Model definitions & stuff

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)  
        self.pool = nn.MaxPool2d(2, 2) 
        self.fc1 = nn.Linear(32 * 128 * 128, 16)
        self.fc2 = nn.Linear(16, 2)
        
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))  
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x)) 
        
        return x

cnn = CNN().to(device)

In [32]:
import torch.optim as optim

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr = 0.001)

#### Model training

In [38]:
batch_size = 16
num_epochs = 10

for epoch in range(num_epochs):
    cnn.train()
    total_loss = 0

    for i in range(0, len(X_train), batch_size):
        batch_data = X_train[i:i + batch_size]  
        batch_labels = y_train[i:i + batch_size]

        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device).long()

        optimizer.zero_grad()

        outputs = cnn(batch_data)
        loss = criterion(outputs, batch_labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(X_train):.4f}')

Epoch [1/10], Loss: 0.0446
Epoch [2/10], Loss: 0.0440
Epoch [3/10], Loss: 0.0453
Epoch [4/10], Loss: 0.0439
Epoch [5/10], Loss: 0.0443
Epoch [6/10], Loss: 0.0446
Epoch [7/10], Loss: 0.0445
Epoch [8/10], Loss: 0.0444
Epoch [9/10], Loss: 0.0444
Epoch [10/10], Loss: 0.0443


In [39]:
def calculate_accuracy(model, X, y, device, batch_size = 16):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            batch_data = X[i:i + batch_size]
            batch_labels = y[i:i + batch_size]
            
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device).long()
            
            outputs = model(batch_data)
            _, predicted = torch.max(outputs, 1)
            
            total += batch_labels.size(0)
            correct += (predicted == batch_labels).sum().item()
    
    return 100 * correct / total

train_acc = calculate_accuracy(cnn, X_train, y_train, device)
test_acc = calculate_accuracy(cnn, X_test, y_test, device)
print(f'Train Accuracy: {train_acc:.2f}%')
print(f'Test Accuracy: {test_acc:.2f}%')

Train Accuracy: 50.00%
Test Accuracy: 100.00%
